In [5]:
import numpy
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import ipykernel
import subprocess
from collections import deque
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
# import gym

In [6]:
print(numpy.version.version)
print(ipykernel.__version__)
tf.config.list_physical_devices()


1.23.3
6.15.2


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# My personal highscore for that level

In [7]:
highscore = {"commands":[{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":4,"rotation":-43},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":3,"rotation":-22},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":4,"rotation":-5},{"power":3,"rotation":0},{"power":3,"rotation":0},{"power":3,"rotation":0},{"power":3,"rotation":0},{"power":3,"rotation":0},{"power":3,"rotation":0},{"power":3,"rotation":0},{"power":3,"rotation":0},{"power":3,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0},{"power":4,"rotation":0}],"score":421}

In [8]:
commands = []
for kv in highscore["commands"]:
  power = kv['power']
  rotation = kv['rotation']
  commands.append(f"{rotation} {power}")

commands[:5]

['-43 4', '-43 4', '-43 4', '-43 4', '-43 4']

In [9]:
pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)

# fullopt

In [10]:
coordinates = df[['x', 'y']].to_numpy()
landing_coordinates = np.array([list(map(int, line.split(' '))) for line in landing_coords] + [[7000, 0], [0, 0]])
poly = Polygon(landing_coordinates, facecolor = 'r')

fig,ax = plt.subplots()
ax.add_patch(poly)
ax.plot(coordinates[:,0], coordinates[:,1]) 
ax.set_xlim([0,7000])
ax.set_ylim([0,3000])
   

NameError: name 'df' is not defined

## Stesstest 100 games
all same seed and the same commands from the highscore path of that level

In [ ]:
%%time
import numpy as np

from MarsLanderGymEnv import MarsLanderGymEnv
import time

n = 100

with MarsLanderGymEnv() as env:

    run_times = []

    start = time.time()

    for episode in range(n):
        s = time.time()
        done = False
        obs = env.reset()
        while True:  # not done:
            random_action = env.action_space.sample()
            # print("action", random_action)
            obs, reward, done, truncated, info = env.step(random_action)
            # print('reward', reward)
            if done:
                # print("is_done")
                break
        e = time.time()
        run_times.append([episode, (e - s) * 1000])

    end = time.time()

    run_times = np.array(run_times)


## looks like hotspot optimization in node vm

In [ ]:
fig,ax = plt.subplots()
display_lim = n
ax.plot(run_times[:display_lim,0], run_times[:display_lim,1])
ax.set_xlabel('run number')
ax.set_ylabel('time (ms)')
ax.set_title(f'runtime over first {display_lim} runs')